In [110]:
# C:\Program Files\Java\jdk-1.8\bin # jvm 경로

In [121]:
# !pip install flask

In [108]:
# !pip install Celery # 비동기을 위한 라이브러리

### 플라스크 성공함

In [1]:
from flask import Flask, request, jsonify
from konlpy.tag import Okt
import jpype
import re

app = Flask(__name__)

@app.route('/', methods=['GET'])
def flask_main():
    print("플라스크 메인 화면 들어옴")
    return "flask main 화면 입니다."

# 텍스트마이닝 요청받았을 때 코드
@app.route('/analyze', methods=['POST'])
def analyze_text():
    print("analyze_text 잘 받아와졌는지 확인")
    okt = Okt()
    print("Okt 초기화 성공")
    # jvm 실행 안됐다면 실행시작
    if not jpype.isJVMStarted():
        jpype.startJVM()

    # node.js 에서 넘긴 데이터 받기
    data = request.json
    user_input = data.get('text', '')
    print("사용자가 작성한 문장 : ", user_input)
    
    # 형태소 쪼개기
    tokens = okt.pos(user_input)
    print("형태소로 쪼개기: ", tokens)

    ########단어 사전 ##########
    # 지역명 단어 사전
    korea_admin_divisions_detailed = {
        "서울특별시": ["서울특별시", "서울"],
        "부산광역시": ["부산광역시", "부산"],
        "대구광역시": ["대구광역시", "대구"],
        "인천광역시": ["인천광역시", "인천"],
        "광주광역시": ["광주광역시", "광주"],
        "대전광역시": ["대전광역시", "대전"],
        "울산광역시": ["울산광역시", "울산"],
        "세종특별자치시": ["세종특별자치시", "세종"],
        "경기도": ["경기도", "경기"],
        "강원도": ["강원도", "강원"],
        "충청북도": ["충청북도", "충북"],
        "충청남도": ["충청남도", "충남"],
        "전라북도": ["전라북도", "전북"],
        "전라남도": ["전라남도", "전남"],
        "경상북도": ["경상북도", "경북"],
        "경상남도": ["경상남도", "경남"],
        "제주특별자치도": ["제주특별자치도", "제주"]
    }

    cities_in_provinces = {
        "서울특별시": ["종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구", "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구", "양천구", "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구", "서초구", "강남구", "송파구", "강동구"],
        "부산광역시": ["중구", "서구", "동구", "영도구", "부산진구", "동래구", "남구", "북구", "해운대구", "사하구", "금정구", "강서구", "연제구", "수영구", "사상구", "기장군"],
        "대구광역시": ["중구", "동구", "서구", "남구", "북구", "수성구", "달서구", "달성군"],
        "인천광역시": ["중구", "동구", "미추홀구", "연수구", "남동구", "부평구", "계양구", "서구", "강화군", "옹진군"],
        "광주광역시": ["동구", "서구", "남구", "북구", "광산구"],
        "대전광역시": ["동구", "중구", "서구", "유성구", "대덕구"],
        "울산광역시": ["중구", "남구", "동구", "북구", "울주군"],
        "세종특별자치시": ["세종시", "세종"],
        "경기도": ["수원시", "수원", "성남시", "성남", "의정부시", "의정부", "안양시", "안양", "부천시", "부천", "광명시", "광명", "평택시", "평택", "동두천시", "동두천", "안산시", "안산", "고양시", "고양", "과천시", "과천", "구리시", "구리", "남양주시", "남양주", "오산시", "오산", "시흥시", "시흥", "군포시", "군포", "의왕시", "의왕", "하남시", "하남", "용인시", "용인", "파주시", "파주", "이천시", "이천", "안성시", "안성", "김포시", "김포", "화성시", "화성", "광주시", "광주", "양주시", "양주", "포천시", "포천", "여주시", "여주", "연천군", "연천", "가평군", "가평", "양평군", "양평"],
        "강원도": ["춘천시", "춘천", "원주시", "원주", "강릉시", "강릉", "동해시", "동해", "태백시", "태백", "속초시", "속초", "삼척시", "삼척", "홍천군", "홍천", "횡성군", "횡성", "영월군", "영월", "평창군", "평창", "정선군", "정선", "철원군", "철원", "화천군", "화천", "양구군", "양구", "인제군", "인제", "고성군", "고성", "양양군", "양양"],
        "충청북도": ["청주시", "청주", "충주시", "충주", "제천시", "제천", "보은군", "보은", "옥천군", "옥천", "영동군", "영동", "증평군", "증평", "진천군", "진천", "괴산군", "괴산", "음성군", "음성", "단양군", "단양"],
        "충청남도": ["천안시", "천안", "공주시", "공주", "보령시", "보령", "아산시", "아산", "서산시", "서산", "논산시", "논산", "계룡시", "계룡", "당진시", "당진", "금산군", "금산", "부여군", "부여", "서천군", "서천", "청양군", "청양", "홍성군", "홍성", "예산군", "예산", "태안군", "태안"],
        "전라북도": ["전주시", "전주", "군산시", "군산", "익산시", "익산", "정읍시", "정읍", "남원시", "남원", "김제시", "김제", "완주군", "완주", "진안군", "진안", "무주군", "무주", "장수군", "장수", "임실군", "임실", "순창군", "순창", "고창군", "고창", "부안군", "부안"],
        "전라남도": ["목포시", "목포", "여수시", "여수", "순천시", "순천", "나주시", "나주", "광양시", "광양", "담양군", "담양", "곡성군", "곡성", "구례군", "구례", "고흥군", "고흥", "보성군", "보성", "화순군", "화순", "장흥군", "장흥", "강진군", "강진", "해남군", "해남", "영암군", "영암", "무안군", "무안", "함평군", "함평", "영광군", "영광", "장성군", "장성", "완도군", "완도", "진도군", "진도", "신안군", "신안"],
        "경상북도": ["포항시", "포항", "경주시", "경주", "김천시", "김천", "안동시", "안동", "구미시", "구미", "영주시", "영주", "영천시", "영천", "상주시", "상주", "문경시", "문경", "경산시", "경산", "군위군", "군위", "의성군", "의성", "청송군", "청송", "영양군", "영양", "영덕군", "영덕", "청도군", "청도", "고령군", "고령", "성주군", "성주", "칠곡군", "칠곡", "예천군", "예천", "봉화군", "봉화", "울진군", "울진", "울릉군", "울릉"],
        "경상남도": ["창원시", "창원", "진주시", "진주", "통영시", "통영", "사천시", "사천", "김해시", "김해", "밀양시", "밀양", "거제시", "거제", "양산시", "양산", "의령군", "의령", "함안군", "함안", "창녕군", "창녕", "고성군", "고성", "남해군", "남해", "하동군", "하동", "산청군", "산청", "함양군", "함양", "거창군", "거창", "합천군", "합천"],
        "제주특별자치도": ["제주시", "제주", "서귀포시", "서귀포"]
    }

    # 질병 단어사전 
    elderly_diseases_and_conditions = [
        "치매", "알츠하이머", "파킨슨병", "관절염", "골다공증", "고혈압", "당뇨병", "심근경색", "뇌졸중", "심부전",
        "백내장", "녹내장", "황반변성", "난청", "귀울림", "저혈압", "폐렴", "기관지염", "COPD", "만성 폐쇄성 폐질환",
        "암", "대장암", "폐암", "위암", "전립선암", "유방암", "간암", "신장암", "췌장암", "갑상선암",
        "통풍", "결석", "신부전", "만성 신부전", "류마티스 관절염", "골관절염", "허리 통증", "요통", "좌골신경통",
        "변비", "설사", "복통", "위염", "소화불량", "장염", "식욕부진", "구토", "구역질", "속쓰림",
        "우울증", "불안장애", "불면증", "기억력 감퇴", "혼란", "피로", "무기력", "우울감", "우울", "외롭", "고립", "고립감", "외로움",
        "피부염", "습진", "건선", "아토피", "피부 건조증", "가려움증", "노인성 소양증", "발진", "종기", "혈관염",
        "부종", "다리 부종", "정맥류", "하지정맥류", "근육통", "근육 경련", "근육 약화", "골절", "낙상", "상처 치유 지연", "손", "발","허리","머리", "거동", "손발"
    ]
    # 장기요양등급 단어사전
    long_term_care_grades = ["1등급", "2등급", "3등급", "4등급", "5등급", "등급 외 A", "등급 외 B"]
    
    # 요일 단어사전
    weekdays = ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일", "월", "화", "수", "목", "금", "토", "일"]
    
    # 시간 단어사전
    times = ["오전", "오후", "저녁", "아침", "밤"]
    
    ######### 단어 사전 끝 #############

    # 정규표현식을 사용하여 나이를 추출하는 함수
    def extract_age(text):
        age_pattern = re.compile(r'(\d+)\s*(세|살)')
        match = age_pattern.search(text)
        if match:
            return match.group(0).replace(" ", "")  # 공백 제거
        return None
    
    # 정규표현식을 사용하여 장기요양등급을 추출하는 함수
    def extract_long_term_care_grade(text):
        grade_pattern = re.compile(r'(\d)등급')
        match = grade_pattern.search(text)
        if match:
            return match.group(1)
        return None


    # 특성을 넣을 빈 배열 선언
    key_info = {
        "age": None,
        "gender": None,
        "disease": None,
        "location": [],
        "long_term_care_grade": None,
        "weekday": None,
        "time": None,
        "additional_info": []
    }
    print("빈배열:", key_info) 
    # 전체 문장에서 나이를 추출하여 key_info에 추가
    key_info["age"] = extract_age(user_input) # 연령

    for word, pos in tokens:
        if word in ["남성", "여성", "남자", "여자", "어머님", "어머니", "시어머니", "엄마","할머니", "할아버지", "아버님", "아빠","아부지","아버지", "할아버지"," 할머님","할아버님"]: # 성별
            key_info["gender"] = word
        elif word in elderly_diseases_and_conditions: # 질병
            key_info["disease"] = word
        elif any(word in names for names in korea_admin_divisions_detailed.values()) or any(word in names for names in cities_in_provinces.values()): # 지역
            key_info["location"].append(word)
        elif word in long_term_care_grades: # 장기요양등급
            key_info["long_term_care_grade"] = word
        elif word in weekdays: # 요일
            key_info["weekday"] = word
        elif word in times: # 시간
            key_info["time"] = word
        else: # 기타 추가 정보
            key_info["additional_info"].append(word)

    key_info["additional_info"] = ' '.join([word for word, pos in tokens if pos not in ['Josa', 'Punctuation']])

    print("결과 : ", key_info)
    return jsonify(key_info)

# 서버 구동 (가장 먼저 실행)
if __name__ == '__main__':
    app.run(host="127.0.0.1", port=5001, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [31/Jul/2024 14:23:20] "GET / HTTP/1.1" 200 -


플라스크 메인 화면 들어옴
analyze_text 잘 받아와졌는지 확인
Okt 초기화 성공
사용자가 작성한 문장 :  치매가 있는 78세 어머니를 병원에 모시기는 싫고 집에 방문해서 주에 2일이라도 좋으니 돌봐주었으면하는데, 그걸 가능하게 하는 기관ㅇ ㅣ있을까? 참고로 요양등급은 3등급을 판정받으셨어.


127.0.0.1 - - [31/Jul/2024 14:23:31] "POST /analyze HTTP/1.1" 200 -


형태소로 쪼개기:  [('치매', 'Noun'), ('가', 'Josa'), ('있는', 'Adjective'), ('78', 'Number'), ('세', 'Noun'), ('어머니', 'Noun'), ('를', 'Josa'), ('병원', 'Noun'), ('에', 'Josa'), ('모', 'Modifier'), ('시기', 'Noun'), ('는', 'Josa'), ('싫고', 'Adjective'), ('집', 'Noun'), ('에', 'Josa'), ('방문', 'Noun'), ('해서', 'Verb'), ('주', 'Noun'), ('에', 'Josa'), ('2일', 'Number'), ('이라도', 'Foreign'), ('좋으니', 'Adjective'), ('돌봐주었으면하는데', 'Verb'), (',', 'Punctuation'), ('그걸', 'Adverb'), ('가능하게', 'Adjective'), ('하는', 'Verb'), ('기관', 'Noun'), ('ㅇ', 'KoreanParticle'), ('ㅣ', 'KoreanParticle'), ('있을까', 'Adjective'), ('?', 'Punctuation'), ('참고', 'Noun'), ('로', 'Josa'), ('요', 'Modifier'), ('양', 'Modifier'), ('등급', 'Noun'), ('은', 'Josa'), ('3등', 'Number'), ('급을', 'Foreign'), ('판정', 'Noun'), ('받으셨어', 'Verb'), ('.', 'Punctuation')]
빈배열: {'age': None, 'gender': None, 'disease': None, 'location': [], 'long_term_care_grade': None, 'weekday': None, 'time': None, 'additional_info': []}
결과 :  {'age': '78세', 'gender': '어머니', 'disease': '치매', 'loca

...

analyze_text 잘 받아와졌는지 확인
Okt 초기화 성공
사용자가 작성한 문장 :  치매가 있는 78세 어머니를 병원에 모시기는 싫고 집에 방문해서 주에 2일이라도 좋으니 돌봐주었으면하는데, 그걸 가능하게 하는 기관ㅇ ㅣ있을까? 참고로 요양등급은 3등급을 판정받으셨어.
형태소로 쪼개기:  [('치매', 'Noun'), ('가', 'Josa'), ('있는', 'Adjective'), ('78', 'Number'), ('세', 'Noun'), ('어머니', 'Noun'), ('를', 'Josa'), ('병원', 'Noun'), ('에', 'Josa'), ('모', 'Modifier'), ('시기', 'Noun'), ('는', 'Josa'), ('싫고', 'Adjective'), ('집', 'Noun'), ('에', 'Josa'), ('방문', 'Noun'), ('해서', 'Verb'), ('주', 'Noun'), ('에', 'Josa'), ('2일', 'Number'), ('이라도', 'Foreign'), ('좋으니', 'Adjective'), ('돌봐주었으면하는데', 'Verb'), (',', 'Punctuation'), ('그걸', 'Adverb'), ('가능하게', 'Adjective'), ('하는', 'Verb'), ('기관', 'Noun'), ('ㅇ', 'KoreanParticle'), ('ㅣ', 'KoreanParticle'), ('있을까', 'Adjective'), ('?', 'Punctuation'), ('참고', 'Noun'), ('로', 'Josa'), ('요', 'Modifier'), ('양', 'Modifier'), ('등급', 'Noun'), ('은', 'Josa'), ('3등', 'Number'), ('급을', 'Foreign'), ('판정', 'Noun'), ('받으셨어', 'Verb'), ('.', 'Punctuation')]
빈배열: {'age': None, 'gender': None, 'disease': None, 'loca

127.0.0.1 - - [31/Jul/2024 14:29:31] "POST /analyze HTTP/1.1" 200 -


analyze_text 잘 받아와졌는지 확인
Okt 초기화 성공
사용자가 작성한 문장 :  치매가 있는 78세 어머니를 병원에 모시기는 싫고 집에 방문해서 주에 2일이라도 좋으니 돌봐주었으면하는데, 그걸 가능하게 하는 기관이 있을까? 참고로 요양등급은 3등급을 판정받으셨어.
형태소로 쪼개기:  [('치매', 'Noun'), ('가', 'Josa'), ('있는', 'Adjective'), ('78', 'Number'), ('세', 'Noun'), ('어머니', 'Noun'), ('를', 'Josa'), ('병원', 'Noun'), ('에', 'Josa'), ('모', 'Modifier'), ('시기', 'Noun'), ('는', 'Josa'), ('싫고', 'Adjective'), ('집', 'Noun'), ('에', 'Josa'), ('방문', 'Noun'), ('해서', 'Verb'), ('주', 'Noun'), ('에', 'Josa'), ('2일', 'Number'), ('이라도', 'Foreign'), ('좋으니', 'Adjective'), ('돌봐주었으면하는데', 'Verb'), (',', 'Punctuation'), ('그걸', 'Adverb'), ('가능하게', 'Adjective'), ('하는', 'Verb'), ('기관', 'Noun'), ('이', 'Josa'), ('있을까', 'Adjective'), ('?', 'Punctuation'), ('참고', 'Noun'), ('로', 'Josa'), ('요', 'Modifier'), ('양', 'Modifier'), ('등급', 'Noun'), ('은', 'Josa'), ('3등', 'Number'), ('급을', 'Foreign'), ('판정', 'Noun'), ('받으셨어', 'Verb'), ('.', 'Punctuation')]
빈배열: {'age': None, 'gender': None, 'disease': None, 'location': [], 'long_term_care_grade': N

127.0.0.1 - - [31/Jul/2024 14:29:45] "POST /analyze HTTP/1.1" 200 -


analyze_text 잘 받아와졌는지 확인
Okt 초기화 성공
사용자가 작성한 문장 :  치매가 있는 78세 어머니를 병원에 모시기는 싫고 집에 방문해서 주에 2일이라도 좋으니 돌봐주었으면하는데, 그걸 가능하게 하는 기관이 있을까? 참고로 요양등급은 3등급을 판정받으셨어.
형태소로 쪼개기:  [('치매', 'Noun'), ('가', 'Josa'), ('있는', 'Adjective'), ('78', 'Number'), ('세', 'Noun'), ('어머니', 'Noun'), ('를', 'Josa'), ('병원', 'Noun'), ('에', 'Josa'), ('모', 'Modifier'), ('시기', 'Noun'), ('는', 'Josa'), ('싫고', 'Adjective'), ('집', 'Noun'), ('에', 'Josa'), ('방문', 'Noun'), ('해서', 'Verb'), ('주', 'Noun'), ('에', 'Josa'), ('2일', 'Number'), ('이라도', 'Foreign'), ('좋으니', 'Adjective'), ('돌봐주었으면하는데', 'Verb'), (',', 'Punctuation'), ('그걸', 'Adverb'), ('가능하게', 'Adjective'), ('하는', 'Verb'), ('기관', 'Noun'), ('이', 'Josa'), ('있을까', 'Adjective'), ('?', 'Punctuation'), ('참고', 'Noun'), ('로', 'Josa'), ('요', 'Modifier'), ('양', 'Modifier'), ('등급', 'Noun'), ('은', 'Josa'), ('3등', 'Number'), ('급을', 'Foreign'), ('판정', 'Noun'), ('받으셨어', 'Verb'), ('.', 'Punctuation')]
빈배열: {'age': None, 'gender': None, 'disease': None, 'location': [], 'long_term_care_grade': N